#001

In [1]:
import numpy as np
import pandas as pd
path = u"./dataset/s2t/jy001.srt"
# path = u"./dataset/001.srt"
x = pd.read_csv(path, sep='\n')
y = np.array(x)

z = []
z.append(np.array(['1']))
for i in y:
    z.append(i)

# for i in z:
#     print(i)

s = ""
idx = 0
subPack = []
pack = []
for i in z:
    subPack.append(i[0])
    idx += 1
    if(idx % 3 == 0):
        pack.append(subPack)
        subPack = []
        idx = 0

start = []
end = []
for i in range(len(pack)):
    s = pack[i][1]
    s = s.replace(" --> ", "")
    s = s[0:8], s[12:20]
    a = list(s)
    pack[i][1] = a


In [2]:
import jieba
import jieba.analyse
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array

config_path = "./model/bert_base/chinese_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/bert_base/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/bert_base/chinese_L-12_H-768_A-12/vocab.txt'

model = build_transformer_model(
    config_path=config_path, checkpoint_path=checkpoint_path, with_nsp=True
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [3]:
pack

[['1',
  ['00:00:02', '00:00:19'],
  '身处分乱之事 心存健康之道 大家好 欢迎来到健康1+1防疫身心灵 我是舅舅 今天是美冬时间2021年11月11号 今天是美国的老兵节 退伍军人节 货兔军人纪念日 那借此机会呢 健康1+1防疫身心灵全体'],
 ['1',
  ['00:00:19', '00:00:39'],
  '红人向退伍军人以及参加过历次战争的退伍军人表达敬意 感谢你们为国家的付出好 那回到疫情的消息 美国cdc本周将欧洲两个国家添加到四级名单 非常高风险的旅游目的名单中截至11月8号cdc的四季名'],
 ['2', ['00:00:39', '00:00:46'], '名单上有80个国家 其中有包括英国 希腊 瑞士 奥地利 土耳其和马尔代夫'],
 ['3',
  ['00:00:47', '00:01:06'],
  '那么的 那疫苗呢 恐发引发心脏病那德国建议呢 30岁以下的人不要使用莫德 那疫苗在美国洛杉矶买菜 还有用餐要看疫苗师打证明 商家如果是违规的话呢 最高要罚13万美元 近期国际权威学术旗杆英国医学'],
 ['4',
  ['00:01:06', '00:01:26'],
  '学旗刊采访了一位协助辉瑞做三起临床试验公司的主任 他指出辉瑞疫苗在临床试验中存在伪造数据 这个采访的镇静医学界 他提出哪些质疑跟证据呢 我们今天将请专家为我们解析 首先还是关注疫情的最新消息'],
 ['5', ['00:01:27', '00:01:33'], 'fax尼夫 法国总统马克隆周二发表电视讲话 但先欧洲目前出现的第五波疫情'],
 ['6', ['00:01:34', '00:01:40'], '如果么在胸的干红不送我 女士们能多多等西东西一大会弄对的'],
 ['7',
  ['00:01:41', '00:01:52'],
  '立着胸送的心牛的了 马克隆敦促人们接种疫苗 他还宣布12月15号以后65岁以上人口必须处事第三季疫苗接种证明健康护照才有效'],
 ['8', ['00:01:53', '00:01:59'], '法国当局还提醒30岁以下人群接种莫德纳疫苗有罹患新基炎和新包炎的风险'],
 ['9', ['00:02:01', '00:02:06'], '莫克 辉瑞等大药厂加速研发 生产抗疫口服药

In [4]:
token = []
seg = []
token_ids = ""
segment_ids = ""
result = []

var = 0.95

for i in range(len(pack)-1):
    if (len(pack[i][2])+len(pack[i+1][2]) >= 512):
        pack[i][2] = pack[i][2][-128:]

    token_ids, segment_ids = tokenizer.encode(
        pack[i][2], pack[i+1][2])

    token_ids, segment_ids = to_array([token_ids], [segment_ids])

    probas = model.predict([token_ids, segment_ids])[0]
    # print(probas)
    result.append(probas)

for i in range(len(pack)-1):
    pack[i].append(result[i][0])

idxList = []
lessThanHalf = []
lessThanHalf.append('00:00:00')

for i in range(len(pack)-1):
    if(pack[i][3] < var):
        idxList.append(i)
        lessThanHalf.append(pack[i][1][1]) #段落起始時間


In [5]:
pack[0]

['1',
 ['00:00:02', '00:00:19'],
 '身处分乱之事 心存健康之道 大家好 欢迎来到健康1+1防疫身心灵 我是舅舅 今天是美冬时间2021年11月11号 今天是美国的老兵节 退伍军人节 货兔军人纪念日 那借此机会呢 健康1+1防疫身心灵全体',
 0.99999464]

In [6]:
combineText = []
j = ''
j = pack[0][2] #段落文本
for i in range(len(pack)-1):
    # token_testId, Seg_testId = tokenizer.encode(pack[i][2], pack[i+1][2])
    # token_testId, Seg_testId = to_array([token_testId], [Seg_testId])
    # result = model.predict([token_testId, Seg_testId])[0][0]

    if(pack[i][3] > var):
        j += pack[i+1][2]
        if(i == len(pack)-2):
            j = pack[i+1][2]
            combineText.append(j)
            j = ""
    else:
        j += pack[i+1][2]

        combineText.append(j)
        j = ""


In [7]:
aa = ""
bb = []
for i in range(len(pack)-1):
    # print(pack[i][-1])

    if len(aa) == 0:
        aa = pack[i][2]

    if (pack[i][3] >= var):
        aa += pack[i+1][2]
    else:
        bb.append(aa)
        aa = ""

    if (i == len(pack)-2):
        # print(0)
        if len(aa) != 0:
            bb.append(aa)
        else:
            bb.append(pack[i+1][2])
combineText = bb


In [8]:
combineText

['身处分乱之事 心存健康之道 大家好 欢迎来到健康1+1防疫身心灵 我是舅舅 今天是美冬时间2021年11月11号 今天是美国的老兵节 退伍军人节 货兔军人纪念日 那借此机会呢 健康1+1防疫身心灵全体红人向退伍军人以及参加过历次战争的退伍军人表达敬意 感谢你们为国家的付出好 那回到疫情的消息 美国cdc本周将欧洲两个国家添加到四级名单 非常高风险的旅游目的名单中截至11月8号cdc的四季名名单上有80个国家 其中有包括英国 希腊 瑞士 奥地利 土耳其和马尔代夫那么的 那疫苗呢 恐发引发心脏病那德国建议呢 30岁以下的人不要使用莫德 那疫苗在美国洛杉矶买菜 还有用餐要看疫苗师打证明 商家如果是违规的话呢 最高要罚13万美元 近期国际权威学术旗杆英国医学学旗刊采访了一位协助辉瑞做三起临床试验公司的主任 他指出辉瑞疫苗在临床试验中存在伪造数据 这个采访的镇静医学界 他提出哪些质疑跟证据呢 我们今天将请专家为我们解析 首先还是关注疫情的最新消息fax尼夫 法国总统马克隆周二发表电视讲话 但先欧洲目前出现的第五波疫情',
 '如果么在胸的干红不送我 女士们能多多等西东西一大会弄对的立着胸送的心牛的了 马克隆敦促人们接种疫苗 他还宣布12月15号以后65岁以上人口必须处事第三季疫苗接种证明健康护照才有效法国当局还提醒30岁以下人群接种莫德纳疫苗有罹患新基炎和新包炎的风险莫克 辉瑞等大药厂加速研发 生产抗疫口服药 各国争相购买台湾委副部长陈实忠周三表示 一经和墨客签约 购买上万份抗议口服药莫纳皮拉维也正在解释辉瑞的资料 韩国一项墨客 辉瑞等药厂预购40 4万份口服药 预计明年二月陆续交货不对呀 大家去了拿一袋mm六鸡蛋 泰国内阁会议周二通过 计划购买5万份 也就是200万片莫克口服药美国合集采购总数310万份 总价22亿美元 英国是全球第一个批准墨客口服药的国家一订购48万份莫克口服药一及25万份灰蕊型药 据称临床数据显示确诊出奇 使用墨克口服药莫纳皮拉维住院或死亡风险可降低一半 灰瑞口服药还在实验中同时是为期待推出鼻喷或口服形式的第二代疫苗',
 '给力你弄就还 反正偶尔把心里弄的你转你是吧 一直也只六个两年你就算了',
 '随着疫苗普及及治疗药物的进展 印度尼西亚和马来西亚两国领导人周三同意在两国之间建立旅游走廊 将首先在首都和度假胜地巴厘岛等地实施',
 '好的 今天的

In [9]:
wst = [] #分詞後文本
for sentence in combineText:
    seq_list = jieba.lcut(sentence) 
    wst.append(seq_list)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SELAB-~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.958 seconds.
Prefix dict has been built successfully.


In [10]:
TF = np.zeros((np.shape(wst)[0], 20)) #產生0陣列 np.shape(TF) = (len(wst), 20)

combineStr = "".join(combineText)

#取詞頻前20的名詞 TFIDF 
Query = jieba.analyse.extract_tags(
    combineStr, topK=20, withWeight=False, allowPOS=('n')) 

for i in range(len(Query)): #Query循環
    for j in range(len(wst)): #wst循環
        for k in range(len(wst[j])): #找出wst[j]中出現了那些top20keyWord
            if(Query[i] == wst[j][k]):
                TF[j][i] = 1
                continue
# print(TF)
keyWord = []
deliver = []
for i in TF:
    for j in range(len(i)):
        if(i[j]):
            deliver.append(Query[j])
    keyWord.append(deliver)
    deliver = []

finalResult = []
tmpProcess = []
for i in range(len(combineText)):
    tmpProcess.append(lessThanHalf[i])
    # tmpProcess.append(lessThanHalf[i+1])
    tmpProcess.append(combineText[i])
    tmpProcess.append(keyWord[i])
    finalResult.append(tmpProcess)
    tmpProcess = []


In [12]:
import jieba
import jieba.analyse
import numpy as np

In [13]:
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array

config_path = "./model/bert_base/chinese_L-12_H-768_A-12/bert_config.json"
checkpoint_path = './model/bert_base/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = './model/bert_base/chinese_L-12_H-768_A-12/vocab.txt'

model = build_transformer_model(
    config_path, 
    checkpoint_path
)
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [14]:
def Euclidean_Distance(v1, v2): #歐式距離
    return np.sqrt(np.sum((v1-v2)**2))


In [15]:
SS=""

for i in range(len(finalResult)):
    SS=SS.join(finalResult[i][2]) #出現的top20keyWord
    print(SS)
    token_ids, segment_ids = tokenizer.encode(SS)
    token_ids, segment_ids = to_array([token_ids], [segment_ids])  
    finalResult[i].append(model.predict([token_ids, segment_ids])[0][0])
    #print(model.predict([token_ids, segment_ids])[0][0])
    SS=""

疫苗数据公司大家疫情临床试验
疫苗数据大家口服药

疫苗药物
疫苗问题数据事情公司情况过程医药公司细胞方面大家报告疫情病人实验者因子药物纰漏临床试验


In [16]:
import math
dist_result=[]
for i in range(len(finalResult)-1):
    finalResult[i].append(Euclidean_Distance(finalResult[i][3],finalResult[i+1][3]))

In [17]:
#dist_result=dist_result[:-1]#沒東西可比 拿掉最後一個
for i in range(len(finalResult)-1):
    dist_result.append(finalResult[i][-1])

In [18]:
trueFinalResult = []
trueText = ""
tureTime = ''
trueKW = []
first = 0
for i in range(len(finalResult)-1):
    #print(trueFinalResult)
    if len(trueText) == 0:
        tureTime = finalResult[i][0]
        trueText = finalResult[i][1]
        trueKW = finalResult[i][2]

    if (finalResult[i][-1] <= 10 or len(finalResult[i][2])==0):
        trueText += finalResult[i+1][1]
        trueKW = list(set(trueKW).union(set(finalResult[i+1][2])))
        #print(tureTime, trueText, trueKW, first)
    else:
        trueFinalResult.append([tureTime,
                                trueText,
                                trueKW
                                ])
        trueText = ""
        tureTime = ''
        trueKW = []
        #print(tureTime, trueText, trueKW, first)
    
    if i==len(dist_result)-1:
        if len(trueText) != 0:
            trueFinalResult.append([tureTime,
                                trueText,
                                trueKW
                                ])
                #print(tureTime, trueText, trueKW, first)
        else:
            print(4)
            tureTime = finalResult[i+1][0]
            trueText = finalResult[i+1][1]
            trueKW = finalResult[i+1][2]
            trueFinalResult.append([tureTime,
                                trueText,
                                trueKW
                                ])
            trueText = ""
            tureTime = ''
            trueKW = []
                #print(tureTime, trueText, trueKW, first)
    
    
# fi    rst=dist_result[i]
# se    cond=dist_result[i+1]
# if    (first<second and first*1.25>second):
#         trueText+=finalResult[i][1]+finalResult[i+1][1]
#         trueFinalResult.append(finalResult[i][0])
#         trueFinalResult.append(trueText)
    

4


In [19]:
list(set(finalResult[0][2]).union(set(finalResult[1][2])))

['临床试验', '口服药', '数据', '疫情', '公司', '疫苗', '大家']

In [20]:
len(trueFinalResult)

4

In [28]:
trueFinalResult

[['00:00:00',
  '身处分乱之事 心存健康之道 大家好 欢迎来到健康1+1防疫身心灵 我是舅舅 今天是美冬时间2021年11月11号 今天是美国的老兵节 退伍军人节 货兔军人纪念日 那借此机会呢 健康1+1防疫身心灵全体红人向退伍军人以及参加过历次战争的退伍军人表达敬意 感谢你们为国家的付出好 那回到疫情的消息 美国cdc本周将欧洲两个国家添加到四级名单 非常高风险的旅游目的名单中截至11月8号cdc的四季名名单上有80个国家 其中有包括英国 希腊 瑞士 奥地利 土耳其和马尔代夫那么的 那疫苗呢 恐发引发心脏病那德国建议呢 30岁以下的人不要使用莫德 那疫苗在美国洛杉矶买菜 还有用餐要看疫苗师打证明 商家如果是违规的话呢 最高要罚13万美元 近期国际权威学术旗杆英国医学学旗刊采访了一位协助辉瑞做三起临床试验公司的主任 他指出辉瑞疫苗在临床试验中存在伪造数据 这个采访的镇静医学界 他提出哪些质疑跟证据呢 我们今天将请专家为我们解析 首先还是关注疫情的最新消息fax尼夫 法国总统马克隆周二发表电视讲话 但先欧洲目前出现的第五波疫情',
  ['疫苗', '数据', '公司', '大家', '疫情', '临床试验']],
 ['00:01:33',
  '如果么在胸的干红不送我 女士们能多多等西东西一大会弄对的立着胸送的心牛的了 马克隆敦促人们接种疫苗 他还宣布12月15号以后65岁以上人口必须处事第三季疫苗接种证明健康护照才有效法国当局还提醒30岁以下人群接种莫德纳疫苗有罹患新基炎和新包炎的风险莫克 辉瑞等大药厂加速研发 生产抗疫口服药 各国争相购买台湾委副部长陈实忠周三表示 一经和墨客签约 购买上万份抗议口服药莫纳皮拉维也正在解释辉瑞的资料 韩国一项墨客 辉瑞等药厂预购40 4万份口服药 预计明年二月陆续交货不对呀 大家去了拿一袋mm六鸡蛋 泰国内阁会议周二通过 计划购买5万份 也就是200万片莫克口服药美国合集采购总数310万份 总价22亿美元 英国是全球第一个批准墨客口服药的国家一订购48万份莫克口服药一及25万份灰蕊型药 据称临床数据显示确诊出奇 使用墨克口服药莫纳皮拉维住院或死亡风险可降低一半 灰瑞口服药还在实验中同时是为期待推出鼻喷或口服形式的第二代疫苗',
  ['疫苗', '数据', '大家', '口服药']],
 ['00:03:00',

In [22]:
len()

TypeError: len() takes exactly one argument (0 given)

In [ ]:
finalResult

[['00:00:00',
  '身处分乱之事 心存健康之道 大家好 欢迎来到健康1+1防疫身心灵 我是舅舅 今天是美冬时间2021年11月11号 今天是美国的老兵节 退伍军人节 货兔军人纪念日 那借此机会呢 健康1+1防疫身心灵全体红人向退伍军人以及参加过历次战争的退伍军人表达敬意 感谢你们为国家的付出好 那回到疫情的消息 美国cdc本周将欧洲两个国家添加到四级名单 非常高风险的旅游目的名单中截至11月8号cdc的四季名名单上有80个国家 其中有包括英国 希腊 瑞士 奥地利 土耳其和马尔代夫那么的 那疫苗呢 恐发引发心脏病那德国建议呢 30岁以下的人不要使用莫德 那疫苗在美国洛杉矶买菜 还有用餐要看疫苗师打证明 商家如果是违规的话呢 最高要罚13万美元 近期国际权威学术旗杆英国医学学旗刊采访了一位协助辉瑞做三起临床试验公司的主任 他指出辉瑞疫苗在临床试验中存在伪造数据 这个采访的镇静医学界 他提出哪些质疑跟证据呢 我们今天将请专家为我们解析 首先还是关注疫情的最新消息fax尼夫 法国总统马克隆周二发表电视讲话 但先欧洲目前出现的第五波疫情',
  ['疫苗', '数据', '公司', '大家', '疫情', '临床试验'],
  array([ 3.11349988e-01,  2.04432920e-01, -4.99550998e-02, -7.25849390e-01,
          1.10270250e+00, -1.17989436e-01, -5.24710059e-01, -8.03674757e-01,
         -3.78470451e-01,  3.85174394e-01, -1.88560739e-01, -4.36205566e-02,
          9.14778948e-01, -1.58704090e+00,  7.11089492e-01,  8.88761818e-01,
          8.26722682e-01, -1.36724174e+00, -6.33268595e-01, -1.56699181e-01,
         -5.99429011e-01,  1.24262519e-01,  4.29374307e-01, -8.81528437e-01,
          4.2281